In [3]:
import ply.lex as lex
import ply.yacc as yacc

# Token list for lexer
tokens = (
    'ID', 'NUMBER',
    'PLUS', 'MINUS', 'TIMES', 'DIVIDE',
    'LPAREN', 'RPAREN',
    'LBRACE', 'RBRACE',
    'SEMI', 'EQUAL',
    'IF', 'ELSE', 'WHILE', 'FOR',
    'INT', 'FLOAT', 'VOID'
)

# Keywords in C++
keywords = {
    'if': 'IF',
    'else': 'ELSE',
    'while': 'WHILE',
    'for': 'FOR',
    'int': 'INT',
    'float': 'FLOAT',
    'void': 'VOID'
}

# Token regex rules
t_PLUS = r'\+'
t_MINUS = r'-'
t_TIMES = r'\*'
t_DIVIDE = r'/'
t_LPAREN = r'\('
t_RPAREN = r'\)'
t_LBRACE = r'\{'
t_RBRACE = r'\}'
t_SEMI = r';'
t_EQUAL = r'='

# Ignored characters (spaces and tabs)
t_ignore = ' \t'

# Identifier (variable names)
def t_ID(t):
    r'[a-zA-Z_][a-zA-Z_0-9]*'
    t.type = keywords.get(t.value, 'ID')  # Check for reserved keywords
    return t

# Number (integer or float)
def t_NUMBER(t):
    r'\d+'
    t.value = int(t.value)
    return t

# Newline tracking
def t_newline(t):
    r'\n+'
    t.lexer.lineno += len(t.value)

# Error handling
def t_error(t):
    print(f"Illegal character '{t.value[0]}'")
    t.lexer.skip(1)

# Build lexer with explicit module reference
lexer = lex.lex(module=__name__)

# Parsing rules
precedence = (
    ('left', 'PLUS', 'MINUS'),
    ('left', 'TIMES', 'DIVIDE')
)

# Symbol table (for tracking variables)
symbol_table = {}

# Function to handle variable declarations
def p_declaration(p):
    '''declaration : type ID SEMI'''
    symbol_table[p[2]] = None
    p[0] = f'{p[2]} = None'

# Type handling (e.g., int, float)
def p_type(p):
    '''type : INT
            | FLOAT
            | VOID'''
    p[0] = p[1]

# Assignment statement
def p_assignment(p):
    '''statement : ID EQUAL expression SEMI'''
    if p[1] in symbol_table:
        p[0] = f'{p[1]} = {p[3]}'
    else:
        raise Exception(f"Undefined variable '{p[1]}'")

# Expressions (handling arithmetic)
def p_expression_binop(p):
    '''expression : expression PLUS expression
                  | expression MINUS expression
                  | expression TIMES expression
                  | expression DIVIDE expression'''
    p[0] = f'({p[1]} {p[2]} {p[3]})'

# Expressions (handling numbers)
def p_expression_number(p):
    '''expression : NUMBER'''
    p[0] = str(p[1])

# Handling if-else statements
def p_if_statement(p):
    '''statement : IF LPAREN expression RPAREN LBRACE statements RBRACE'''
    p[0] = f'if {p[3]}:\n' + indent(p[6])

def p_if_else_statement(p):
    '''statement : IF LPAREN expression RPAREN LBRACE statements RBRACE ELSE LBRACE statements RBRACE'''
    p[0] = f'if {p[3]}:\n' + indent(p[6]) + '\nelse:\n' + indent(p[10])

# Handling while loop
def p_while_statement(p):
    '''statement : WHILE LPAREN expression RPAREN LBRACE statements RBRACE'''
    p[0] = f'while {p[3]}:\n' + indent(p[6])

# Statement list
def p_statements(p):
    '''statements : statement
                  | statement statements'''
    p[0] = p[1] + '\n' + (p[2] if len(p) > 2 else '')

# Error handling
def p_error(p):
    if p:
        print(f"Syntax error at '{p.value}'")
    else:
        print("Syntax error at EOF")

# Helper function to indent code blocks
def indent(code):
    return '\n'.join(['    ' + line for line in code.splitlines() if line])

# Build the parser
parser = yacc.yacc()

# Function to convert C++ code to Python
def cpp_to_python(cpp_code):
    result = parser.parse(cpp_code)
    return result

# Read from input.txt and write


KeyError: '__module__'